In [1]:
#https://github.com/aspiringfastlaner/crypto_testing/blob/master/coinmarketcaps.ipynb

import re
import time
import datetime as dt
import pandas as pd
import numpy as np
import json
import requests
import csv
import os
import glob
from bs4 import BeautifulSoup as bs
from pandas import ExcelWriter

def save_xls(list_dfs, xls_path, sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

In [5]:
'''
Fields
market_cap_usd
name
price_usd
symbol
'''
exclusion_list = []# ['BCC','XRP','USDT']
capital = 20000

response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
json_data = json.loads(response.text)
i = 0
crypto_df = {}
for cc in json_data:
    temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
    if len(crypto_df) == 0:
        crypto_df = temp_df
    else:
        crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
    i += 1
    
crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False]#[:30]

crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
crypto_df['dollar_weight'] = crypto_df['weight']*capital
crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']

crypto_list = [(lambda x: x.lower().replace(" ", "-"))(x) for x in list(crypto_df['name'])]

In [6]:
# Starting table: <table class="table">
# Ending table: </table>

# site: https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20171111

ticker = 'bitcoin'
start_date = '20130428'
end_date = '20171111'

def coinmktcapPull(ticker, start_date, end_date):
    
    wrongticker = "Sorry, we couldn't find your page"

    date_range = '/historical-data/?start=' + start_date + '&end=' + end_date
    site = 'https://coinmarketcap.com/currencies/' + ticker + date_range
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    #soup = soup.prettify()

    if wrongticker in soup:
        return ticker, ticker

    table_start = '<table class="table">'
    table_end = '</table>'
    if len(soup.find_all('table')) == 0:
        return ticker, ticker

    table = soup.find_all('table')[0]    

    # Parsing table from beautiful soup text
    row_marker = 0
    dates = []
    opens = []
    highs = []
    lows = []
    closes = []
    volumes = []
    mktcaps = []
    for row in table.find_all('tr'):
        column_marker = 0
        columns = str(row).split('\n')
        if 'Date' in columns[1]:
            continue
        else:
            dates = dates + [columns[1].split('<td class="text-left">')[1].split('</td>')[0]]
            opens = opens + [columns[2].split('<td>')[1].split('</td>')[0]]
            highs = highs + [columns[3].split('<td>')[1].split('</td>')[0]]
            lows = lows + [columns[4].split('<td>')[1].split('</td>')[0]]
            closes = closes + [columns[5].split('<td>')[1].split('</td>')[0]]
            volumes = volumes + [columns[6].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
            mktcaps = mktcaps + [columns[7].split('<td>')[1].split('</td>')[0].replace(',','').replace('-','0')]
    '''
    crypto_dict = {ticker + '_open': opens,
                   ticker + '_high': highs,
                   ticker + '_low': lows,
                   ticker + '_close': closes,
                   ticker + '_volume': volumes,
                   ticker + '_mktcap': mktcaps}
    '''
    crypto_closes = {ticker: closes} #{ticker + '_close': closes}
    crypto_mktcaps = {ticker: mktcaps}#{ticker + '_mktcap': mktcaps}
    close_df = pd.DataFrame(crypto_closes, index = dates)
    mktcap_df = pd.DataFrame(crypto_mktcaps, index = dates)
    
    #coinmktcapdf[ticker + '_open'] = pd.to_numeric(coinmktcapdf[ticker + '_open'])
    #coinmktcapdf[ticker + '_high'] = pd.to_numeric(coinmktcapdf[ticker + '_high'])
    #coinmktcapdf[ticker + '_low'] = pd.to_numeric(coinmktcapdf[ticker + '_low'])
    close_df[ticker] = pd.to_numeric(close_df[ticker])
    #coinmktcapdf[ticker + '_volume'] = pd.to_numeric(coinmktcapdf[ticker + '_volume'])
    mktcap_df[ticker] = pd.to_numeric(mktcap_df[ticker])
    close_df.index = pd.to_datetime(close_df.index)
    mktcap_df.index = pd.to_datetime(mktcap_df.index)
    return close_df, mktcap_df

In [7]:
all_close = []
all_mktcap = []
wrong_names = []

start_date = '20130428'
end_date = '20171112'
for tick in crypto_list:
    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
    if type(temp_df) == str:
        wrong_names += [temp_close]
    else:
        all_close += [temp_close]
        all_mktcap += [temp_mktcap]
        
correct_names = ['stellar', 'bytecoin-bcn','metaverse','byteball','attention-token-of-media','rlc']

for tick in correct_names:
    temp_close, temp_mktcap = coinmktcapPull(tick, start_date, end_date)
    if type(temp_df) == str:
        wrong_names += [temp_close]
    else:
        all_close += [temp_close]
        all_mktcap += [temp_mktcap]

In [8]:

cc_close = pd.concat(filter(lambda x: type(x) != str, all_close), axis = 1).fillna(0)
cc_mcaps = pd.concat(filter(lambda x: type(x) != str, all_mktcap), axis = 1).fillna(0)

cc_index = cc_close[['bitcoin']]
cc_index.columns = ['index']
cc_returns = cc_close.pct_change().fillna(0)

In [9]:
i = 0
top_picks = 5
last_month = int(cc_index.index[i].month)
index_start = 100

for idx, row in cc_index.iterrows():
    curr_month = int(idx.month)
    if i == 0:
        mcaps_temp = cc_mcaps.iloc[i].sort_values(ascending = False)[:top_picks]
        mcaps_temp = mcaps_temp[mcaps_temp != 0]
        wts = mcaps_temp/mcaps_temp.sum()
    elif curr_month != last_month:
        mcaps_temp = cc_mcaps.iloc[i].sort_values(ascending = False)[:top_picks]
        mcaps_temp = mcaps_temp[mcaps_temp != 0]
        wts = mcaps_temp/mcaps_temp.sum()
    else:
        pass
    
    temp_index = pd.concat([cc_returns.iloc[i],wts],axis = 1).dropna()
    temp_index.columns = ['returns','weights']
    idx_return = (temp_index['returns']*temp_index['weights']).sum()
    
    idx_val = idx_return
    
    cc_index.set_value(idx,'index', idx_val)
    
    last_month = curr_month
    i += 1
    
cc_index = cc_index.replace(np.inf, np.nan).fillna(0)

In [10]:
def getccwts(exclusions, capital, top_n):
    
    response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
    json_data = json.loads(response.text)
    i = 0
    crypto_df = {}
    for cc in json_data:
        temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
        if len(crypto_df) == 0:
            crypto_df = temp_df
        else:
            crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
        i += 1

    crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
    crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

    crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
    crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False][:top_n]

    crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
    crypto_df['dollar_weight'] = crypto_df['weight']*capital
    crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']
    
    return crypto_df

In [17]:
exclusion_list = ['BCC','XRP','USDT','XEM','ETC']
capital = 5000
top_n = 10

test = getccwts(exclusion_list, capital, top_n)
test['price_eth'] = test['price_usd']/315.55
test['eth_cost'] = test['price_eth']*test['expected_holdings']
test

,symbol,name,market_cap_usd,price_usd,weight,dollar_weight,expected_holdings,price_eth,eth_cost
0,BTC,Bitcoin,1.122009e+11,6727.360000,0.632158,3160.789406,0.469841,21.319474,10.016762
1,ETH,Ethereum,3.047352e+10,318.390000,0.171693,858.463204,2.696263,1.009000,2.720530
2,BCH,Bitcoin Cash,2.136805e+10,1271.820000,0.120391,601.955093,0.473302,4.030486,1.907638
4,LTC,Litecoin,3.295314e+09,61.231400,0.018566,92.831607,1.516078,0.194047,0.294190
5,DASH,Dash,3.201817e+09,416.522000,0.018040,90.197720,0.216550,1.319987,0.285843
6,XMR,Monero,1.886817e+09,122.953000,0.010631,53.153134,0.432304,0.389647,0.168446
7,NEO,NEO,1.858564e+09,28.593300,0.010471,52.357240,1.831102,0.090614,0.165924
10,MIOTA,IOTA,1.604292e+09,0.577181,0.009039,45.194184,78.301579,0.001829,0.143224
11,QTUM,Qtum,8.293818e+08,11.259700,0.004673,23.364345,2.075042,0.035683,0.074043
12,OMG,OmiseGO,7.700906e+08,7.546760,0.004339,21.694066,2.874620,0.023916,0.068750


In [18]:
L = []
L.append(test)
save_xls(L,"coinmarketcap.xlsx",['Sheet1'])